## Start with the necessary imports

In [ ]:
import os

import nipype
import nibabel as nib
import numpy as np
from nilearn.plotting import plot_roi
from nilearn import plotting

from nimare.dataset import Dataset
from nimare.decode import discrete
from nimare.utils import get_resource_path

## Load dataset with abstracts

In [ ]:
out_dir = "/XXX/FunctionalDecoding/example_data/neurosynth_LDA50/"
dset = Dataset.load(os.path.join(out_dir, "neurosynth_dataset_LDA50.pkl.gz"))

## Decode an ROI image using the Neurosynth ROI association method

In [ ]:
#Load file
rootdir = '/XXX/FunctionalDecoding/'
files=['ROI_056_SalVentAttn_L_PI_RO.nii','ROI_069_SalVentAttn_R_MCC_MPC.nii','ROI_074_SalVentAttn_R_AI.nii']
filenames=['L_PI_RO','R_MCC_MPC','R_AI']
out_dir = "/XXX/FunctionalDecoding/"


decoded_all={}
for file,name in zip(files,filenames):

    # Load images
    img = nib.load(os.path.join(rootdir, file)) 
    img = img.get_fdata()

    # Orientation of the images is flipped
    img=np.flip(img, axis=0)
    arr = np.zeros(dset.masker.mask_img.shape, int)
    arr[img>0] = 1
    mask_img = nib.Nifti1Image(arr, dset.masker.mask_img.affine)

    # Get studies with voxels in the mask
    ids = dset.get_studies_by_mask(mask_img)

    # Run the decoder
    decoder_roiassociation = discrete.ROIAssociationDecoder(mask_img)
    decoder_roiassociation.fit(dset)

    # The `transform` method doesn't take any parameters.
    decoded_roiassociation_df = decoder_roiassociation.transform()

    decoded_roiassociation_df.sort_values(by="r", ascending=False).head()

    csvfile=out_dir + "decoder_roiassociation_" + name + ".csv"
    decoded_roiassociation_df.to_csv(csvfile)
